# DONT COMMIT THE FILE

# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


### First, download the data from: https://www.kaggle.com/ntnu-testimon/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?

In [1]:
# Your code here
import pandas as pd
paysim = pd.read_csv("PS_20174392719_1491204439457_log.csv")

### What is the distribution of the outcome? 

In [2]:
# Your response here
paysim.hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x0000016019DB8D68>,
      dtype=object)

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [3]:
paysim.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [32]:
# Your code here
paysim.dtypes

step                       int64
type                      object
amount                   float64
oldbalanceOrg            float64
newbalanceOrig           float64
oldbalanceDest           float64
newbalanceDest           float64
isFraud                    int64
isFlaggedFraud             int64
time              datetime64[ns]
dtype: object

In [31]:
### The step should be displayed as an time series. But we only have the information that the time period is a month.
import datetime



paysim["time"] = pd.to_datetime(paysim["step"])

In [11]:
#Drop the name columns because they have to many unique values
paysim = paysim.drop(["nameOrig", "nameDest"], axis =1 )

In [33]:
#now that we have the time column we can drop the step one
paysim = paysim.drop("step", axis = 1)

In [36]:
#now we need to deal with the type column.

paysim_modif = pd.concat([paysim, paysim["type"].str.get_dummies(" , ")], axis=1)
paysim_modif.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,time,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0,1970-01-01 00:00:00.000000001,0,0,0,1,0
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0,1970-01-01 00:00:00.000000001,0,0,0,1,0
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0,1970-01-01 00:00:00.000000001,0,0,0,0,1
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0,1970-01-01 00:00:00.000000001,0,1,0,0,0
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0,1970-01-01 00:00:00.000000001,0,0,0,1,0


In [37]:
paysim_modif = paysim_modif.drop("type", axis = 1)

In [43]:
from datetime import datetime as dt

def convert_date_to_ordinal(date):
    return date.toordinal()


paysim_modif["time"] = paysim_modif["time"].apply((lambda x: convert_date_to_ordinal(x))) 

### Run a logisitc regression classifier and evaluate its accuracy.

In [44]:
# Your code here
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = paysim_modif.drop("isFraud", axis = 1)
y =paysim_modif["isFraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=43)
lr = LogisticRegression() ##inicialization
lr.fit(X_train,y_train)
acc = lr.score(X_test,y_test)*100    

C:\Users\mari_\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
acc

99.92408787574931

### Now pick a model of your choice and evaluate its accuracy.

In [ ]:
# Your code here
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = knn.score(X_test, y_test)*100
print(f"2 neighbors KNN Score: {round(acc,2)}%")


### Which model worked better and how do you know?

In [2]:
# Your response here

